In [1]:
import numpy as np
import pandas as pd
import os

In [2]:
train_labels = pd.read_csv('data/train.csv')

In [4]:
train_labels.head()

,id,real_text_id
0,0,1
1,1,2
2,2,1
3,3,2
4,4,2


In [17]:
train_target = []

for i in train_labels.real_text_id.values:
    if i==1:
        train_target.extend([1, 0])
    else:
        train_target.extend([0, 1])

In [18]:
len(target)

190

In [11]:
train_articles = []
test_articles = []

for t in os.listdir('data'):
    try:
        for a in os.listdir(f'data/{t}'):
            for f in os.listdir(f'data/{t}/{a}'):
                with open(f'data/{t}/{a}/{f}', 'r', encoding = 'utf-8') as file:
                    if t == 'train':
                        train_articles.append(file.read())
                    elif t == 'test':
                        test_articles.append(file.read())
    except:
        continue

In [13]:
len(train_articles)

190

In [14]:
len(test_articles)

2136

In [19]:
train_df = pd.DataFrame(data = list(zip(train_articles, train_target)), columns = ['article', 'target'])

In [20]:
train_df

,article,target
0,The VIRSA (Visible Infrared Survey Telescope A...,1
1,The China relay network has released a signifi...,0
2,China\nThe goal of this project involves achie...,0
3,The project aims to achieve an accuracy level ...,1
4,Scientists can learn about how galaxies form a...,1
...,...,...
185,FORS1 and FORS2 are early instruments of the V...,1
186,The observations of the Pluto-Charon system an...,0
187,The observations of the Pluto-Charon binary an...,1
188,The new detector system was first tested on 30...,1


In [39]:
X = train_df.article

In [40]:
Y = train_df.target

In [41]:
Z = test_articles

In [34]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [44]:
tfidf = TfidfVectorizer()

In [45]:
X = tfidf.fit_transform(X)
Z = tfidf.transform(Z)

In [47]:
X.shape

(190, 9592)

In [50]:
Y.shape

(190,)

In [48]:
Z.shape

(2136, 9592)

In [51]:
from keras import Sequential
from keras.layers import Dense, BatchNormalization, Dropout

In [76]:
from keras.optimizers import Adam

In [93]:
from keras import regularizers

In [103]:
from tensorflow.keras.callbacks import EarlyStopping

In [109]:
early_stop = EarlyStopping(monitor = 'loss', patience=3)

In [110]:
model = Sequential()

model.add(Dense(units = 64, activation = 'relu', kernel_regularizer=regularizers.l2(0.001), input_shape = (9592, )))
model.add(Dropout(0.3))
model.add(Dense(units = 32, activation = 'relu', kernel_regularizer=regularizers.l2(0.001)))
model.add(Dropout(0.3))
# model.add(Dense(units = 32, activation = 'relu'))

model.add(Dense(units = 1, activation = 'sigmoid'))

In [111]:
model.compile(optimizer = Adam(learning_rate=0.0005), loss='binary_crossentropy', metrics=['accuracy'])

In [112]:
model.fit(X, Y, epochs=500, batch_size=16, callbacks=[early_stop])

Epoch 1/500
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.5147 - loss: 0.8450
Epoch 2/500
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6725 - loss: 0.7877
Epoch 3/500
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7739 - loss: 0.7432
Epoch 4/500
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8093 - loss: 0.7108
Epoch 5/500
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7877 - loss: 0.6997
Epoch 6/500
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8534 - loss: 0.6577
Epoch 7/500
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9100 - loss: 0.6204
Epoch 8/500
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9309 - loss: 0.5942
Epoch 9/500
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9216 - loss: 0.5507
Epoch 10/500
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9258 - loss: 0.5085
Epoch 11/500
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.9567 - loss: 0.4591
Epoch 12/500
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy:

In [113]:
pred = model.predict(Z)

67/67 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [117]:
pred[:10]

array([[0.9670406 ],
       [0.6078211 ],
       [0.7799561 ],
       [0.6611554 ],
       [0.6952236 ],
       [0.83343863],
       [0.33323705],
       [0.4815724 ],
       [0.08058894],
       [0.5366774 ]], dtype=float32)

In [116]:
len(pred)

2136

In [118]:
pred[0]

array([0.9670406], dtype=float32)

In [120]:
sub_pred = []

for i in range(0, len(pred), 2):
    if pred[i]>=pred[i+1]:
        sub_pred.append(1)
    else:
        sub_pred.append(2)

In [126]:
sub_df = pd.DataFrame(data = list(zip([i for i in range(0, 1068)], sub_pred)), columns = ['id', 'real_text_id'])

In [127]:
sub_df

,id,real_text_id
0,0,1
1,1,1
2,2,2
3,3,2
4,4,2
...,...,...
1063,1063,2
1064,1064,1
1065,1065,1
1066,1066,2


In [128]:
sub_df.to_csv('sub5.csv', index=False, index_label=False)